In [ ]:
!pip install google-cloud-secret-manager
!pip install --upgrade google-cloud-aiplatform
!pip install google-cloud-secret-manager

In [ ]:
import sys
!{sys.executable} -m pip install langchain huggingface_hub

!{sys.executable} -m  pip install bitsandbytes>=0.39.0
!{sys.executable} -m pip install --upgrade accelerate
!{sys.executable} -m pip install --upgrade sentence_transformers
!{sys.executable} -m  pip install --upgrade transformers
!{sys.executable} -m pip install --upgrade trl
!{sys.executable} -m pip install --upgrade peft
!{sys.executable} -m pip install --upgrade sqlite3
!{sys.executable} -m pip install --upgrade pytz
!{sys.executable} -m pip install --upgrade pypdf PyPDF2
!{sys.executable} -m pip install --upgrade langchain_experimental
!{sys.executable} -m pip install --upgrade langchain_community
!{sys.executable} -m pip install --upgrade langchain_openai
!{sys.executable} -m pip install --upgrade openai tiktoken chromadb
!pip install unstructured['pdf'] unstructured
!{sys.executable} -m pip install --upgrade textwrap
!pip install --upgrade --quiet  cohere
!pip install langchain-cohere
!pip install grandalf
!pip install wikipedia
!pip install faiss-cpu
!pip install faiss-gpu
!pip install langchainhub
!pip install -qU langchain-mistralai
!pip install "unstructured[html]"
!pip install bs4
!pip install rank_bm25
!pip install gradio
!pip install langchain_chroma



import gradio as gr
from huggingface_hub import InferenceClient

import os
import time
import sqlite3
import re
from pytz import timezone
import pytz
import datetime
import json
import textwrap
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
from operator import itemgetter
import os
import sys

import peft
import bitsandbytes
import accelerate
import trl
from accelerate import init_empty_weights
import torch
import pandas as pd

from transformers import AutoConfig, BitsAndBytesConfig
from accelerate.utils import BnbQuantizationConfig, load_and_quantize_model
from accelerate import load_checkpoint_and_dispatch
import accelerate, bitsandbytes

from langchain_mistralai import ChatMistralAI
import getpass
from google.colab import userdata


from langchain import hub
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from langchain_experimental.agents import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import UnstructuredPDFLoader, PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_community.document_loaders import BSHTMLLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import WebBaseLoader
from langchain.vectorstores import Chroma

from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

from langchain.chains import RetrievalQA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from langchain.llms import HuggingFaceHub
from langchain_community.llms import Cohere
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.callbacks.tracers import ConsoleCallbackHandler

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import AgentType, initialize_agent
from langchain_core.messages import HumanMessage, ToolMessage, SystemMessage, AIMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from langchain import PromptTemplate, FewShotPromptTemplate

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from google.colab import userdata


In [ ]:
from google.colab import userdata
from google.cloud import aiplatform
from sentence_transformers import SentenceTransformer
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models
from vertexai.language_models import TextGenerationModel, ChatModel, InputOutputTextPair

import getpass
from google.colab import userdata
import base64
import vertexai
from typing import List, Optional
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel
from vertexai.generative_models import GenerativeModel,GenerationConfig, Part, FinishReason, FunctionDeclaration,Tool,HarmBlockThreshold
from vertexai.generative_models import HarmCategory, grounding, GenerationResponse
import vertexai.preview.generative_models as generative_models
from vertexai.preview.generative_models import grounding as preview_grounding
from google.cloud import aiplatform

**Embeddings model**

In [ ]:
model_name = "Alibaba-NLP/gte-large-en-v1.5"
model_kwargs = {'device': 'cpu', 'trust_remote_code': True}
encode_kwargs = {'normalize_embeddings': False}

hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)




**Vector Search endpoint**

In [ ]:
endpoint = "projects/930596367380/locations/us-central1/indexEndpoints/6368806754683191296" # copy fullqualified name from console
new_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(endpoint)


**Function to return sources**

In [ ]:
def filenameRegistry(name):
  name = name.split('/')[0]
  gcsname_to_filename = {'cliguide_chunks':"Alteon CLI User's Guide.pdf"}
  gcsname_to_link = {'cliguide_chunks':"https://storage.googleapis.com/bucker_uscentral/cliguide_chunks/Alteon CLI User's Guide.pdf"}
  try:
    return [gcsname_to_filename[name], gcsname_to_link[name]]
  except:
    return ['error retrieving filename','error retrieving filename' ]


**Gemini LLM**

In [ ]:

MODEL_ID = "gemini-1.5-flash-001"
model = GenerativeModel(MODEL_ID)


generation_config = GenerationConfig(
    temperature=0,
    top_p=0.95,
    top_k=40,
    candidate_count=1, #The number of response variations to return. This value must be 1
    max_output_tokens=8192,
)

nosafetysettings = {
#    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE, # only for monthly invoiced billed account
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
}


def llmgem(prompt):
  response = model.generate_content(
    prompt, # contents can be supplied here as well which is prompt sent as a single text element in a list
    generation_config=generation_config,
    safety_settings=nosafetysettings,
)
  return response.text

**Function for semantic search using vertex ai vector search and retriver of chunks**

In [ ]:
import concurrent.futures
from google.cloud import storage
import io

def download_blob_to_memory(blob_name, bucket_name ="bucker_uscentral"):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    in_memory_file = io.BytesIO()
    blob.download_to_file(in_memory_file)
    in_memory_file.seek(0)
    return blob_name, in_memory_file

def downloadFilesMemory(blob_name, bucket_name ="bucker_uscentral"):

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    contents = blob.download_as_bytes()

    return blob_name, contents.decode("utf-8")

def download_files_concurrently(file_list,bucket_name ="bucker_uscentral", max_workers=5):

    with concurrent.futures.ThreadPoolExecutor(max_workers= max_workers) as executor:
        futures = [
            executor.submit(download_blob_to_memory, file_name,bucket_name  )
            for file_name in file_list
        ]

        results = {}
        for future in concurrent.futures.as_completed(futures):
            blob_name, in_memory_file = future.result()
            results[blob_name] = in_memory_file

    return results



class vsearchretriever (object):
  def __init__(self,index_endpoint, index_id = "index_dims1024_stream",num_neighbors=10):
    self.index_endpoint = index_endpoint
    self.index_id = index_id
    self.num_neighbors = num_neighbors
    self.cites = []

  def _get_vec_neighbors(self, query):
    embeddings = hf.embed_query(query)
    self.response = self.index_endpoint.find_neighbors(
        deployed_index_id=self.index_id ,
        queries=[embeddings],
        num_neighbors= self.num_neighbors,
    )
    return self.response

  def get_docs(self, query):
    self.response = self._get_vec_neighbors(query)
    dictforcontent = {'id':[],'distance':[]}
    for i in self.response[0]:
      dictforcontent['id'].append(i.id)
      dictforcontent['distance'].append(i.distance)
    orderedids = dictforcontent['id']
    newdict = {}
    self.context = []
    self.cites = []
    downloaded_files = download_files_concurrently(dictforcontent['id'])
    for blob_name, in_memory_file in downloaded_files.items():
        content = in_memory_file.getvalue().decode("utf-8")
        newdict[blob_name] = Document(page_content=content, metadata={"source": filenameRegistry(blob_name)[0]})
        self.cites.append(filenameRegistry(blob_name)[0])
    for i in orderedids:
      self.context.append(newdict[i]) # added for langc documents

    self.cites = list(set(self.cites)) # remove duplicates

    return self.context # for LC we need to return a list [Documents(),..]


**Instantiate BM25 for hybrid retreival and semantic search retriever using above function**

In [ ]:

retrieverbm25 = BM25Retriever.from_texts(["Arnab says to enable data port with management access use /c/sys/ports/ena"])
retrieverbm25.k = 4
retriever = vsearchretriever(new_index_endpoint,index_id = "index_dims1024_stream", num_neighbors=10)
retrieverembed = retriever.get_docs

**Final RAG pipeline with everything in a langchain chain**

In [ ]:

##### template ####

template = """
Answer the following question based only on the provided context:

<context>

{context}

</context>

Question: {query}"""

prompt = PromptTemplate(
    input_variables=["context", "query"],
    template=template,
)


def lcprompt(prompt):
  return prompt.to_string()

def format_chunks(docs):
  """
  Format the chunks for the prompt context
  """
  return "\n\n".join(doc.page_content for doc in docs)

compressor = CohereRerank(model="rerank-english-v3.0",top_n=7) # topn 5 was producing better results on some questions

#### ensemble retriever for semantic and keyword search ###########
ensemble_retriever_cliguide = EnsembleRetriever(retrievers=[RunnableLambda(retrieverembed),retrieverbm25],weights=[0.8, 0.2])

######## cohere reranker#####################
compression_retriever_cliguide = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble_retriever_cliguide
)

#### langchain LCEL ###########################
setupandret = RunnableParallel({"context": compression_retriever_cliguide | format_chunks , "query": RunnablePassthrough()})
chain_setup = setupandret
chain_prompt = setupandret | prompt | RunnableLambda(lcprompt)
chain = setupandret | prompt | RunnableLambda(lcprompt) | RunnableLambda(llmgem)
query ='How to disable management access to data ports ?'
print ("query: ", query)
print (chain.invoke(query) + "\n\nSources:\n\n" + "\n".join(retriever.cites))


